### 2-4-LiveBeeBackgroundRemoval.ipynb

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import sys
import glob
import os
import csv
import cv2
import time
import warnings

from PIL import Image
from pathlib import Path
from tqdm import tqdm

from scipy.spatial import cKDTree
from scipy.ndimage import binary_fill_holes
from scipy.ndimage import label, sum as ndimage_sum

from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

In [2]:
# Define directories
data_dir = Path("/mnt/g/Projects/Master/Data/")

input_dir = data_dir / "Processed" / "LiveBees" / "4-LiveWingCropsManuallyImproved" 
output_dir = data_dir / "Processed" / "LiveBees" / "5-LiveWingCropsRemovedBackground" 
output_wing_dir = output_dir / "Wings"

# Sam checkpoint path
sam2_checkpoint = "/home/wsl/bin/segment-anything-2/checkpoints/sam2_hiera_tiny.pt"
model_cfg = "sam2_hiera_t.yaml"

DEBUG = False

In [7]:
def remove_points_near_border(points, contour, border_dist_threshold):
    """
    Filters out points too close to the edge of the wing to improve segment anything results.
    """
    filtered_points = []

    # Iterate over all points
    for point in points:
        # Check the distance of the point to the contour
        dist_to_contour = cv2.pointPolygonTest(contour, (point[0], point[1]), True)
        
        # Keep the point if it's farther from the border than the threshold
        if dist_to_contour >= border_dist_threshold:
            filtered_points.append(point)
    
    return np.array(filtered_points)
        

def postprocess_mask(mask):
    """
    Improves the output created by segment anything by selecting the best mask and filling holes.
    """
    labeled_mask, num_features = label(mask)
    if num_features == 0: 
        return mask
    component_sizes = ndimage_sum(mask, labeled_mask, range(1, num_features + 1))
    largest_component_label = np.argmax(component_sizes) + 1 
    largest_component_mask = labeled_mask == largest_component_label

    largest_component_mask = np.squeeze(largest_component_mask)
    filled_image = binary_fill_holes(largest_component_mask)

    clean_mask = binary_fill_holes(largest_component_mask)
    if DEBUG:
        # Display original and filled images
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        axes[0].imshow(largest_component_mask, cmap="gray")
        axes[0].set_title("largest_component_mask")
        axes[0].axis("off")
        axes[1].imshow(clean_mask, cmap="gray")
        axes[1].set_title("clean_mask")
        axes[1].axis("off")
        plt.show()
    return clean_mask


def remove_background(wing, t=120):
    """
    Uses segment anything to create a wing mask and removes everything outside the mask.
    """
    expanded_image = wing
    # Convert the image to grayscale
    gray = cv2.cvtColor(expanded_image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding to get a binary image
    _, thresh = cv2.threshold(gray, t, 255, cv2.THRESH_BINARY)
    
    # Invert the binary image
    inv_thresh = cv2.bitwise_not(thresh)
    
    # Find contour
    contours, _ = cv2.findContours(inv_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 5000]
    
    # Find the largest contour based on area
    contour = max(large_contours, key=cv2.contourArea)
    
    # Draw contours on the image for visualization
    wing_contour_image = expanded_image.copy()
    cv2.drawContours(wing_contour_image, large_contours, -1, (0, 0, 255), 5)

    if DEBUG:
        # Show image
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(wing_contour_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
    
    # Get bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Create a dense grid of points within the bounding box
    distance = 200  
    height, width, channels = expanded_image.shape  
    
    # Create x and y coordinates
    x_coords = np.arange(0, width, distance)
    y_coords = np.arange(0, height, distance)
    
    # Create a meshgrid from the x and y coordinates
    grid_x, grid_y = np.meshgrid(x_coords, y_coords)
    
    # Stack the x and y coordinates into a single array of points
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T
    
    # Convert the NumPy array to a list of tuples with standard integers
    grid_points = [(int(x), int(y)) for x, y in grid_points]
    
    inside_points = []
    
    # Check if points are inside the contour
    for point in grid_points:
        if cv2.pointPolygonTest(contour, (point[0], point[1]), False) >= 0:
            inside_points.append(point)
    
    filtered_points = remove_points_near_border(inside_points, contour, 25)

    if DEBUG:
        # Plot the contour and the selected points
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(wing_contour_image, cv2.COLOR_BGR2RGB))
        plt.scatter(filtered_points[:, 0], filtered_points[:, 1], c="red", s=5)
        plt.axis('off')
        plt.show()
    
    # Sam background removal
    # Convert the points list to a numpy array
    image_points = np.array(filtered_points)
    image_labels = np.array([1] * len(filtered_points))
    
    predictor.set_image(expanded_image)
    
    mask, score, _ = predictor.predict(
        point_coords=image_points,
        point_labels=image_labels,
        multimask_output=False,
    )
    sorted_ind = np.argsort(score)[::-1]
    mask = mask[sorted_ind]
    score = score[sorted_ind]

    # Improve mask quality
    mask = postprocess_mask(mask)
    
    # Remove extra dimension
    mask = mask.squeeze()
    
    # Create a white image of the same size as the original image
    white_image = np.ones_like(expanded_image) * 255
    
    # Apply the mask to each channel (no extra dimension added)
    wing_image = np.where(mask[:, :, None], expanded_image, white_image)

    if DEBUG:
        # Show image
        plt.figure(figsize=(20, 20))
        plt.imshow(cv2.cvtColor(wing_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

    return wing_image

def crop_wing(wing_image):
    """
    This main function finds an initial wing contour which is then improved with segment anything to remove the background. 
    """
    expanded_image = cv2.copyMakeBorder(wing_image, 1000, 1000, 1000, 1000, cv2.BORDER_CONSTANT, value=[255, 255, 255])
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(expanded_image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding to get a binary image
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    
    # Invert the binary image
    inv_thresh = cv2.bitwise_not(thresh)

    # Find contour
    contours, _ = cv2.findContours(inv_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 5000]

    # Find the largest contour based on area
    contour = max(large_contours, key=cv2.contourArea)
    
    # Draw contours on the image for visualization
    wing_contour_image = expanded_image.copy()
    cv2.drawContours(wing_contour_image, large_contours, -1, (0, 0, 255), 5)

    # Get the minimum area rectangle
    rect = cv2.minAreaRect(contour)
    
    # Get the four points of the rectangle
    box = cv2.boxPoints(rect)
    
    # Convert the points to integers
    box = np.intp(box)
    
    # Draw the rotated rectangle
    contour_image = expanded_image.copy()
    cv2.drawContours(contour_image, [box], 0, (0, 0, 255), 5)
    
    # Get the rectangle's center, size (width, height), and angle
    box_center, box_size, angle = rect
    
    # Ensure width is the longest side (width > height)
    width, height = box_size
    if height > width:
        width, height = height, width
        angle -= 90  # Rotate to make the longest side horizontal
    
    # Get the rotation matrix to rotate the image around the rectangle's center
    rotation_matrix = cv2.getRotationMatrix2D(box_center, angle, 1.0)
    
    # Rotate the entire image
    rotated_image = cv2.warpAffine(expanded_image, rotation_matrix, (expanded_image.shape[1], expanded_image.shape[0]))
    
    # Convert the center and size to integers
    box_center = (int(box_center[0]), int(box_center[1]))
    width = int(width) 
    height = int(height)
    
    # Crop the aligned rectangle from the rotated image
    cropped_image = cv2.getRectSubPix(rotated_image, (width+100, height+100), box_center)

    return(cropped_image, height)

In [8]:
# Ignore segment-anything warnings
warnings.filterwarnings("ignore")

# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

# Load sam model
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)
predictor = SAM2ImagePredictor(sam2_model)

try:
    # Create the new output directories
    if os.path.exists(output_wing_dir):
        print("WARNING: Output directory already exists.") 
    os.makedirs(output_wing_dir, exist_ok=True)

    # Create or open a CSV file and write the header if it doesn't exist
    output_file_path = os.path.join(output_dir, "WingHeight.csv")
    header_written = os.path.exists(output_file_path)
    with open(output_file_path, mode="a", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["Filename", "WingHeightInPixels"])
        if not header_written:
            writer.writeheader()

        # Process every jpg file
        jpg_files = list(input_dir.glob("*.JPG"))
        for jpg_file in tqdm(jpg_files, desc="Processing files", ncols=145):
            increase_tresh = False
            filename = jpg_file.name
            output_file = output_wing_dir / filename
        
            # Skip if the file exists
            if os.path.exists(output_file):
                continue
    
            # Open the image
            wing = Image.open(jpg_file)
            wing = np.array(wing.convert("RGB"))
            brightness = np.mean(wing)
            # print(f"brightness: {brightness}")
            if brightness > 180:
                increase_tresh = True
            else:
                try:    
                    new_wing = remove_background(wing, 120)
                    cropped_image, height_pixels = crop_wing(new_wing)
                    # print(f"height_pixels: {height_pixels}")
                except (IndexError, ValueError):
                    increase_tresh = True
            if height_pixels < 200 or increase_tresh:
                wing = Image.open(jpg_file)
                wing = np.array(wing.convert("RGB"))
                new_wing = remove_background(wing, 200)
                cropped_image, height_pixels = crop_wing(new_wing)
                
            # Save the image and wing height
            cropped_image = Image.fromarray(cropped_image)
            cropped_image.save(output_file)
            writer.writerow({"Filename": filename, "WingHeightInPixels": height_pixels})
        
# Handle exceptions
except FileNotFoundError as e:
    print(e)

except KeyboardInterrupt:
    pass

using device: cpu


Processing files: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1194/1194 [39:02<00:00,  1.96s/it]
